# 📘 Classificador de Gatos vs Cachorros com Keras
Este notebook combina o treinamento e avaliação de um modelo de deep learning com TensorFlow/Keras para classificar imagens de gatos e cachorros.

In [ ]:
# 🧰 Importações
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import numpy as np
import os

## ⚙️ Configurações
Define os parâmetros principais como o tamanho da imagem, batch size, número de épocas e diretórios dos dados e modelo.

In [ ]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 50
DATA_DIR = 'data'
MODEL_PATH = 'dogs_vs_cats_model.keras'

## 📂 Preparação dos Dados
Cria geradores de imagens para os conjuntos de treino e validação com aumentação de dados para melhorar a generalização.

In [ ]:
def prepare_data():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_set = train_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'training_set'),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )

    validation_set = validation_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'validation_set'),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )

    return train_set, validation_set

## 🏗️ Construção do Modelo
Define a arquitetura da rede neural convolucional com camadas de convolução, pooling, dropout e densas.

In [ ]:
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(*IMG_SIZE, 3)),
        MaxPool2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPool2D((2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

## 🚂 Treinamento do Modelo
Treina o modelo usando os dados preparados e salva o melhor modelo com early stopping. Também plota gráficos de acurácia e perda.

In [ ]:
def train():
    train_set, validation_set = prepare_data()
    model = build_model()

    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(
        train_set,
        steps_per_epoch=train_set.samples // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=validation_set,
        validation_steps=validation_set.samples // BATCH_SIZE,
        callbacks=[early_stop]
    )

    model.save(MODEL_PATH)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Treino')
    plt.plot(history.history['val_accuracy'], label='Validação')
    plt.title('Acurácia por Época')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Treino')
    plt.plot(history.history['val_loss'], label='Validação')
    plt.title('Loss por Época')
    plt.legend()

    plt.savefig('training_metrics.png')
    plt.show()

## 🧪 Avaliação com Conjunto de Teste
Avalia o modelo treinado usando o conjunto de teste e mostra a acurácia, perda e o relatório de classificação.

In [ ]:
def evaluate_model():
    model = load_model(MODEL_PATH)

    test_datagen = ImageDataGenerator(rescale=1./255)
    test_set = test_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'test_set'),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )

    loss, accuracy = model.evaluate(test_set)
    print(f'\nAcurácia no Teste: {accuracy:.4f}')
    print(f'Loss no Teste: {loss:.4f}\n')

    y_pred = (model.predict(test_set) > 0.5).astype(int)
    print(classification_report(
        test_set.classes,
        y_pred,
        target_names=test_set.class_indices.keys()
    ))

## 🔍 Predição Individual de Imagens
Classifica imagens individuais como gato ou cachorro com base no modelo salvo.

In [ ]:
def predict_image(image_path):
    model = load_model(MODEL_PATH)
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)[0][0]
    class_name = 'Cachorro' if pred > 0.5 else 'Gato'
    confidence = pred if pred > 0.5 else 1 - pred

    print(f'\nImagem: {os.path.basename(image_path)}')
    print(f'Predição: {class_name} (Confiança: {confidence:.2%})')

In [ ]:
# ▶️ Executar avaliação e predições (certifique-se de que o modelo e imagens estejam no local correto)
evaluate_model()

predict_image('images/gato1.jpg')  
predict_image('images/gato2.jpg')
predict_image('images/gato3.jpg') 
predict_image('images/cachorro1.jpg')
predict_image('images/cachorro2.jpg')
predict_image('images/cachorro3.jpg')